In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
%matplotlib inline

In [2]:
from sklearn.linear_model import ElasticNet, Lasso,  Ridge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

/Users/kobayashihiroki/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
train = pd.read_csv('train_data_2.csv')
test = pd.read_csv('test_data_2.csv')

In [4]:
train_x = train.drop(['SalePrice'], axis=1)
train_y = train['SalePrice']
test_x = test

## モデル作成
1. ラッソ回帰
2. リッジ回帰
3. ElasticNet (リッジ回帰とラッソ回帰をうまく組み合わせたもの)
4. ランダムフォレスト
5. Gradient Boosting Regression
6. XGBoost
7. LightGBM
8. Neural Network
9. n-近傍法

### 下準備

In [5]:
from sklearn import linear_model

In [6]:
from sklearn.preprocessing import RobustScaler

#### RobustScaler (外れ値に強い標準化)

In [7]:
rs = RobustScaler()
train_x_rs = rs.fit_transform(train_x)

#### 評価精度の良いパラメータを探す

In [8]:
lasso_cv = linear_model.LassoCV(alphas=[0.00001, 0.0001, 0.0004, 0.001, 0.01], cv=5)
lasso_cv.fit(train_x_rs, train_y)
print("alpha = ", lasso_cv.alpha_)

alpha =  0.0004


In [9]:
reg_cv = linear_model.RidgeCV(alphas=[0.1, 1, 7, 10, 100], cv=5)
reg_cv.fit(train_x_rs, train_y)
print("alpha = ", reg_cv.alpha_)

alpha =  7.0


In [10]:
ElasticNet_cv = linear_model.ElasticNetCV(alphas=[0.0001, 0.0005, 0.001, 0.01], l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=5)
ElasticNet_cv.fit(train_x_rs, train_y)
print("alpha = ", ElasticNet_cv.alpha_)
print("l1_ratio = ", ElasticNet_cv.l1_ratio_)

/Users/kobayashihiroki/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.050274063060278706, tolerance: 0.017997180045467572
  tol, rng, random, positive)
/Users/kobayashihiroki/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.05917487904554619, tolerance: 0.01837752313707959
  tol, rng, random, positive)
/Users/kobayashihiroki/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.024602631401936748, tolerance: 0.017997180045467572
  tol, rng, random, positive)


alpha =  0.0005
l1_ratio =  0.7


### ①ラッソ回帰のモデルを作成

In [11]:
from sklearn.pipeline import make_pipeline

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
def rmsle_cv(model):
    #kf = KFold(n_splits=5, shuffle=True, random_state=42).get_n_splits()
    rmse= np.sqrt(-cross_val_score(model, train_x.values, train_y.values, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [14]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha = 0.0004, random_state=51))

In [15]:
print('Lasso score: {:.4f}'.format(rmsle_cv(lasso).mean()))

Lasso score: 0.1133


### ②リッジ回帰のモデルを作成

In [16]:
ridge = make_pipeline(RobustScaler(), Ridge(alpha = 7.0, random_state=51))

In [17]:
print('Ridge score:{:.4f}'.format(rmsle_cv(ridge).mean()))

Ridge score:0.1152


### ③ElasticNetのモデルの作成

In [18]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha = 0.0005, l1_ratio=0.7, random_state=51))   # l1_ratio・・l1ノルムの割合

In [19]:
print('ElasticNet score: {:.4f}'.format(rmsle_cv(ENet).mean()))

ElasticNet score: 0.1133


### ④ランダムフォレストのモデル

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
param_grid = {
    'n_estimators' : [200, 250],
    'max_depth' : [20, 25],
    'min_samples_leaf' : [3, 5],
    'min_samples_split' : [4, 6]
}

grid = GridSearchCV(RandomForestRegressor(), param_grid, scoring= 'neg_mean_squared_error', cv=5)
grid.fit(train_x, train_y)
print(grid.best_params_, np.sqrt(-grid.best_score_))

{'max_depth': 20, 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 250} 0.1394349656142441


In [22]:
randomforest = RandomForestRegressor(max_depth=20, min_samples_leaf=3, min_samples_split=6, n_estimators=250,
                                    random_state=51)

In [23]:
rmsle_cv(randomforest).mean()

0.13967290711021194

### PCA分析で次元を減らし、ランダムフォレストしたモデル

In [24]:
from sklearn.decomposition import PCA

In [26]:
pipe = make_pipeline(StandardScaler(), PCA(), RandomForestRegressor())

param_grid = {
    'pca__n_components': [8, 16],
    'randomforestregressor__n_estimators' : [200, 250],
    'randomforestregressor__max_depth' : [20, 25],
    'randomforestregressor__min_samples_leaf' : [3, 5],
    'randomforestregressor__min_samples_split' : [4, 6]
}

grid = GridSearchCV(pipe, param_grid, scoring= 'neg_mean_squared_error', cv=5)
grid.fit(train_x, train_y)
print(grid.best_params_, np.sqrt(-grid.best_score_))

{'pca__n_components': 16, 'randomforestregressor__max_depth': 20, 'randomforestregressor__min_samples_leaf': 3, 'randomforestregressor__min_samples_split': 4, 'randomforestregressor__n_estimators': 200} 0.15976973491266766


In [27]:
PCA_randomforest = make_pipeline(RobustScaler(), PCA(n_components=16, random_state=51), 
                                 RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_leaf=3,
                                                       min_samples_split=4, random_state=51))

In [28]:
rmsle_cv(PCA_randomforest).mean()

0.16586585009129878

- PCA分析を挟むと精度が落ちた（StandardScaler( )ではなくRobustScaler( )も試した）

### ⑤Gradient Boosting Regression

In [29]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [30]:
rmsle_cv(GBoost).mean()

0.1182663299106251

### ⑥XGBoost

### 最適なパラメータをoptunaを用いて調べる

In [31]:
import optuna

In [32]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
from sklearn.model_selection import KFold

In [34]:
# Objective Functionの作成
def opt(trial):
    n_estimators = trial.suggest_int('n_estimators', 0, 1000)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    subsample = trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1)
    colsample_bytree = trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1)
    xgboost_tuna = xgb.XGBRegressor(
        random_state=51,
        n_estimators = n_estimators,
        max_depth = max_depth,
        min_child_weight = min_child_weight,
        subsample = subsample,
        colsample_bytree = colsample_bytree,
    )
    
    preds = []
    va_idxes = []
    kf = KFold(n_splits=4, shuffle=True, random_state=51)
    for tr_idx, va_idx in kf.split(train_x):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        
        xgboost_tuna.fit(tr_x, tr_y)
        tuna_va_pred = xgboost_tuna.predict(va_x)
        preds.append(tuna_va_pred)
        va_idxes.append(va_idx)
        
    va_idxes = np.concatenate(va_idxes, axis=0)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]
    return (np.sqrt(mean_squared_error(train_y, pred_train)))
 

In [35]:
# 最適化
study = optuna.create_study()
study.optimize(opt, n_trials=100)

[18:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:11:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:11:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:11:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2020-02-28 18:11:57,354] Finished trial#0 resulted in value: 0.12019200649469368. Current best value is 0.12019200649469368 with parameters: {'n_estimators': 535, 'max_depth': 1, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.9}.


[18:11:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:12:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:12:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:12:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2020-02-28 18:12:44,974] Finished trial#1 resulted in value: 0.12504037645888927. Current best value is 0.12019200649469368 with parameters: {'n_estimators': 535, 'max_depth': 1, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.9}.


[18:12:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:13:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:13:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:13:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[I 2020-02-28 19:01:19,476] Finished trial#97 resulted in value: 0.12302898562891662. Current best value is 0.11724616479844059 with parameters: {'n_estimators': 498, 'max_depth': 2, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.7}.


[19:01:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2020-02-28 19:01:32,637] Finished trial#98 resulted in value: 0.1192952541811095. Current best value is 0.11724616479844059 with parameters: {'n_estimators': 498, 'max_depth': 2, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.7}.


[19:01:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2020-02-28 19:01:55,645] Finished trial#99 resulted in value: 0.12177629353795831. Current best value is 0.11724616479844059 with parameters: {'n_estimators': 498, 'max_depth': 2, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.7}.


In [36]:
# 結果を出力
print(study.best_params)
print(study.best_value)
print(study.best_trial)

{'n_estimators': 498, 'max_depth': 2, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.7}
0.11724616479844059
FrozenTrial(number=61, value=0.11724616479844059, datetime_start=datetime.datetime(2020, 2, 28, 18, 40, 54, 293222), datetime_complete=datetime.datetime(2020, 2, 28, 18, 52, 25, 275620), params={'n_estimators': 498, 'max_depth': 2, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.7}, distributions={'n_estimators': IntUniformDistribution(high=1000, low=0), 'max_depth': IntUniformDistribution(high=20, low=1), 'min_child_weight': IntUniformDistribution(high=20, low=1), 'subsample': DiscreteUniformDistribution(high=0.9, low=0.5, q=0.1), 'colsample_bytree': DiscreteUniformDistribution(high=0.9, low=0.5, q=0.1)}, user_attrs={}, system_attrs={'_number': 61}, intermediate_values={}, trial_id=61, state=TrialState.COMPLETE)


In [37]:
xgboost = xgb.XGBRegressor(
        random_state=51,
        n_estimators = 498,
        max_depth = 2,
        min_child_weight = 3,
        subsample = 0.7,
        colsample_bytree = 0.7)

In [38]:
rmsle_cv(xgboost).mean()

[19:03:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:03:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:03:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:03:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:03:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.11898187485798602

### ⑦LightGBM

In [39]:
lightgbm = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [40]:
rmsle_cv(lightgbm).mean()

0.117162256737917

### ⑧Neural Network

In [41]:
from sklearn.neural_network import MLPRegressor

In [42]:
NeuralNet = MLPRegressor(random_state=51)

In [43]:
rmsle_cv(NeuralNet).mean()

0.1852345003941589

### 隠れ層を増やしてみる

In [44]:
NeuralNet = MLPRegressor(hidden_layer_sizes=(100,100,100,100,100, ), random_state=51)

In [45]:
rmsle_cv(NeuralNet).mean()

0.15536496835948888

### ⑨n-近傍法

In [46]:
from sklearn.neighbors import KNeighborsRegressor

In [47]:
param_grid = {
    'n_neighbors' : [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
}

grid = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5)
grid.fit(train_x, train_y)
print(grid.best_params_, grid.best_score_)

{'n_neighbors': 5} 0.6053381812223322


In [48]:
knn = KNeighborsRegressor(n_neighbors=5)

In [49]:
rmsle_cv(knn).mean()

0.2500294249779272

### スタッキング

In [50]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        # 1層目 学習させて全てのデータに対しての予測ラベルをbase_model個分手に入れる
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        # 2層目 学習して手に入れた予測ラベル(base_model個分)と実際の目的変数とを学習させる
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [51]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, ridge, randomforest, GBoost),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

Stacking Averaged models score: 0.1101 (0.0066)


### 二乗平均平方根誤差 RMSE

In [52]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

#### StackedRegressor

In [53]:
stacked_averaged_models.fit(train_x.values, train_y.values)
stacked_train_pred = stacked_averaged_models.predict(train_x.values)
stacked_pred = np.exp(stacked_averaged_models.predict(test_x.values))
print(rmsle(train_y.values, stacked_train_pred))

0.08151157817684582


#### XGBoost

In [54]:
xgboost.fit(train_x, train_y)
xgb_train_pred = xgboost.predict(train_x)
xgb_pred = np.exp(xgboost.predict(test))
print(rmsle(train_y, xgb_train_pred))

[23:39:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.0715907786165376


#### lightGBM

In [55]:
lightgbm.fit(train_x, train_y)
lgb_train_pred = lightgbm.predict(train_x)
lgb_pred = np.exp(lightgbm.predict(test_x.values))
print(rmsle(train_y, lgb_train_pred))

0.07667811183385176


In [56]:
print('RMSLE score on train data:')
print(rmsle(train_y, stacked_train_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

RMSLE score on train data:
0.07700582157509175


### 加重平均をとって予測値を作成

In [57]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [58]:
ensemble

array([120665.86022304, 155931.78255315, 186022.56521093, ...,
       170448.40077411, 118501.77415445, 225388.45427755])

In [59]:
sub = pd.read_csv('test.csv')
sub

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [60]:
submission = pd.DataFrame({'Id': sub['Id'], 
                           'SalePrice': ensemble})

In [61]:
submission

,Id,SalePrice
0,1461,120665.860223
1,1462,155931.782553
2,1463,186022.565211
3,1464,197054.517951
4,1465,198441.636889
...,...,...
1454,2915,84470.575984
1455,2916,83827.134466
1456,2917,170448.400774
1457,2918,118501.774154


In [62]:
submission.to_csv('submission.csv', index=False)